# Factor-Based Stock Selection vs the S&P 500 (FF5 Rolling Strategy + ML Ranking)

## Notebook flow (baseline + two-model comparison)

This notebook contains:

1. **A transparent baseline strategy** using rolling FF5 betas to form a monthly top‑50 portfolio.
2. A **two-model machine learning comparison** (the part you “tried two models and chose one”):
   - **Model 1:** Linear model (Logistic Regression)
   - **Model 2:** XGBoost learning-to-rank

Both ML models are trained and evaluated walk-forward (no look-ahead), and the final section compares the two models against **SPY**.

### Sections

1. **Setup** (imports + helper functions)
2. **Data preparation**
   - FF5 daily → monthly (compounded)
   - Load S&P 500 monthly returns
   - Merge + excess returns
3. **Baseline factor strategy** (rolling FF5 betas → signal → top‑50 portfolio)
4. **Backtest & benchmarking** (baseline vs SPY)
5. **Two-model ML comparison** (Linear vs XGBoost)
6. **Final conclusion** (choose the better-performing model)

Run the notebook top-to-bottom (Kernel: Restart & Run All) to reproduce results and regenerate figures in `docs/assets/`.

### 1. Import Libraries

We import the core Python libraries needed for this project:
- **pandas** for data loading and cleaning  
- **numpy** for numerical regression  
- **matplotlib** for visualization  
- set the global plotting style for consistency


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

plt.style.use("seaborn-v0_8")

# Ensure docs assets folder exists for GitHub Pages figures
os.makedirs("docs/assets", exist_ok=True)

# XGBoost (Model 2 learning-to-rank)
try:
    from xgboost import XGBRanker
    _HAS_XGBOOST = True
except Exception:
    XGBRanker = None
    _HAS_XGBOOST = False

## Part 0. Setup Helpers

This section defines the small helper functions used throughout the notebook (OLS and rolling FF5 regression). Defining them up front makes the execution flow easier to follow.

In [ ]:
def ols_numpy(X, y):
    """Perform closed-form OLS regression."""
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    return beta


In [ ]:
def rolling_factor_loadings(df, ticker, window=36, min_obs=24):
    betas = []
    dates = []

    y_all = df[ticker].astype(float).values
    X_all = df[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]].astype(float).values
    X_all = np.column_stack([np.ones(len(X_all)), X_all])

    for i in range(window, len(df)):
        y = y_all[i - window:i]
        X = X_all[i - window:i]

        mask = np.isfinite(y) & np.all(np.isfinite(X), axis=1)
        if mask.sum() < min_obs:
            continue

        beta = ols_numpy(X[mask], y[mask])
        betas.append(beta)
        dates.append(df["date"].iloc[i])

    if len(betas) == 0:
        return pd.DataFrame(columns=["alpha", "MKT", "SMB", "HML", "RMW", "CMA"])

    return pd.DataFrame(
        betas,
        index=dates,
        columns=["alpha", "MKT", "SMB", "HML", "RMW", "CMA"],
    )


## Part I. Baseline Strategy (Rolling FF5 Betas → Signal → Top-50 Portfolio)

This part builds a transparent walk-forward factor strategy:
- Estimate rolling 36-month FF5 betas per stock (on excess returns)
- Forecast factor returns using a lagged trailing mean (no look-ahead)
- Rank stocks each month by predicted return signal
- Form a long-only equal-weight top-50 portfolio and compare vs SPY

### 2. Load and Prepare Fama–French 5-Factor Data

The FF5 dataset is provided in **daily format**, so we first:
1. Load the raw daily CSV
2. Remove footer text rows
3. Convert the date to a proper `datetime`
4. Forward-fill missing values
5. Convert from **daily → monthly** by **compounding within each month**
6. Convert the monthly dates to **month-start** to align with our market data

This ensures both datasets use the *same monthly timestamps* and that the monthly factors represent monthly returns (not a single end-of-month observation).

In [ ]:
ff = pd.read_csv("data/raw/ff5_data.csv")
ff = ff[ff["Date"].astype(str).str.isdigit()].copy()
ff["date"] = pd.to_datetime(ff["Date"].astype(str), format="%Y%m%d", errors="coerce")
ff = ff.dropna(subset=["date"]).drop(columns=["Date"]).sort_values("date").reset_index(drop=True)

factor_cols = ["Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"]
ff[factor_cols] = ff[factor_cols].ffill()
ff[factor_cols] = ff[factor_cols] / 100.0

ff = ff.set_index("date")[factor_cols].sort_index()

# Monthly RF by compounding daily RF
rf_monthly = (1 + ff["RF"]).resample("M").prod() - 1

# Monthly Mkt-RF computed from compounded market total return minus compounded RF
mkt_total_daily = ff["Mkt-RF"] + ff["RF"]
mkt_total_monthly = (1 + mkt_total_daily).resample("M").prod() - 1
mkt_rf_monthly = mkt_total_monthly - rf_monthly

# Style factors compounded within the month
other_factors = ["SMB", "HML", "RMW", "CMA"]
other_monthly = (1 + ff[other_factors]).resample("M").prod() - 1

ff_monthly = pd.concat(
    [mkt_rf_monthly.rename("Mkt-RF"), other_monthly, rf_monthly.rename("RF")],
    axis=1,
)

# Align to month-start timestamps (matches market data index)
ff_monthly.index = ff_monthly.index.to_period("M").to_timestamp()
ff_monthly = ff_monthly.reset_index().rename(columns={"date": "date"})

ff_monthly.head()

### 3. Load Market Return Data (Monthly)

We load the pre-cleaned **monthly return matrix** of S&P 500 constituents.
The dataset is already in wide format (`ticker columns`), and dates are already the **first day of each month**, so minimal cleaning is needed.

In [ ]:
market = pd.read_csv("data/raw/market_data.csv", parse_dates=["Date"])
market = market.rename(columns={"Date": "date"})
market = market.sort_values("date").reset_index(drop=True)

market.head()


### 4. Merge Market Returns with Factor Data

We merge the monthly market returns with the aligned monthly FF5 factors.
The merge occurs cleanly after aligning FF5 month-end dates to month-start.

We also fix column naming issues (e.g., `RF_y`) that arise from merging datasets with overlapping column names.

In [ ]:
df = market.merge(ff_monthly, on="date", how="inner")

if "RF_y" in df.columns:
    df = df.rename(columns={"RF_y": "RF"})
if "RF_x" in df.columns:
    df = df.drop(columns=["RF_x"])

factor_cols = ["Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"]
tickers = [c for c in df.columns if c not in ["date"] + factor_cols]

df.head()


### 5. Compute Excess Returns for Each Stock

To prepare the regression inputs, we convert raw stock returns into **excess returns**:

\[
R_{i,t}^{excess} = R_{i,t} - RF_t
\]

This is standard in multi-factor regression based on FF5.


In [ ]:
df_total = df.copy()

df_excess = df.copy()
df_excess[tickers] = df_excess[tickers].sub(df_excess["RF"], axis=0)

### 6. Ordinary Least Squares (OLS) via NumPy

The helper function `ols_numpy(...)` is defined in **Part 0**. It uses `np.linalg.lstsq` to compute OLS coefficients for the rolling regressions.

In [ ]:
pass

### 7. Rolling-Window Factor Regression

The helper function `rolling_factor_loadings(...)` is defined in **Part 0**. For each stock, it fits a 36-month rolling regression of excess returns on the FF5 factors and stores the estimated betas over time.

\[
R^{excess}_{i,t} = \alpha_i + \beta_{i}^T F_t + \epsilon_t
\]


In [ ]:
pass

In [ ]:
len(df)

### 8. Run Rolling-Window Regressions Across All S&P 500 Tickers

We apply the rolling regression to every ticker in the dataset.
Some tickers may be skipped due to insufficient return history (e.g., IPOs).

In [ ]:
beta_dict = {}

for t in tickers:
    print("Running:", t)
    try:
        beta_dict[t] = rolling_factor_loadings(df_excess, t)
    except Exception as e:
        print(f"Skipping {t}: {e}")

### 9. Predict Returns Using Rolling Betas (Signal Generation)

We generate a monthly expected return signal by combining each stock’s rolling factor exposures (betas) with a forecast of factor returns.

Key design choices to improve robustness:
- **No look-ahead**: factor forecasts are lagged by 1 month, so the signal at month \(t\) uses information available through \(t-1\).
- **No intercept (alpha) in the signal**: we exclude regression intercepts from the prediction to reduce overfitting risk.

This produces a cross-sectional expected return signal for each stock that we rank each month.

In [ ]:
factor_hist = df.set_index("date")[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]]

# Forecast factors using only information available before month t (avoid look-ahead)
factor_forecast = factor_hist.rolling(window=36, min_periods=24).mean().shift(1)

predictions = {}

for t in tickers:
    betas = beta_dict.get(t)
    if betas is None or betas.empty:
        continue

    idx = betas.index.intersection(factor_forecast.index)
    if len(idx) == 0:
        continue

    # Use factor exposures only (exclude intercept/alpha to reduce overfitting risk)
    beta_factors = betas.loc[idx, ["MKT", "SMB", "HML", "RMW", "CMA"]].values
    f_mat = factor_forecast.loc[idx, ["Mkt-RF", "SMB", "HML", "RMW", "CMA"]].values

    pred = (beta_factors * f_mat).sum(axis=1)
    predictions[t] = pd.Series(pred, index=idx)

pred_df = pd.DataFrame(predictions)
pred_df.head()

### 10. Long-Only Portfolio Construction (Top 50 Stocks)

Each month:
- Sort predicted returns
- Select the **top 50 stocks**
- Assign equal weights (1/50 each)

This produces a simple, transparent long-only factor strategy.

In [ ]:
def build_long_only_portfolio(pred_df, k=50):
    weights = pd.DataFrame(0.0, index=pred_df.index, columns=pred_df.columns)

    for dt, row in pred_df.iterrows():
        row = row.dropna()
        if row.empty:
            continue

        top_k = row.nlargest(min(k, len(row))).index
        w = 1.0 / len(top_k)
        weights.loc[dt, top_k] = w

    return weights

weights = build_long_only_portfolio(pred_df)
weights.head()


## Part II. Backtest & Benchmarking (Strategy vs SPY)

### 11. Realized Monthly Portfolio Returns

We evaluate performance by applying the portfolio weights formed at month \(t\) to the realized stock returns for month \(t\).

Because the expected return signal at month \(t\) is constructed using only information available up to \(t-1\) (rolling betas and a lagged factor forecast), this is an out-of-sample evaluation at a monthly frequency.

\[
R^{portfolio}_t = \sum_i w_{i,t} R_{i,t}
\]


In [ ]:
# Realized (same-month) total returns for each stock
realized_returns = df_total.set_index("date")[tickers]

common_index = weights.index.intersection(realized_returns.index)
weights_aligned = weights.loc[common_index]
realized_aligned = realized_returns.loc[common_index]

available = realized_aligned.notna()
w_eff = weights_aligned.where(available, 0.0)
denom = w_eff.sum(axis=1).replace(0.0, np.nan)

# Effective portfolio weights actually used after dropping missing returns
weights_used = w_eff.div(denom, axis=0)

portfolio_returns = (w_eff * realized_aligned.fillna(0.0)).sum(axis=1) / denom
active = denom.notna()

portfolio_returns = portfolio_returns.loc[active]
weights_aligned = weights_aligned.loc[active]
weights_used = weights_used.loc[active]

portfolio_returns.head()

---

### 12. Load SPY Benchmark Returns

We load monthly SPY returns to benchmark our strategy.
The benchmark is aligned to the same monthly timestamps as the portfolio.

In [ ]:
benchmark = pd.read_csv("data/raw/spy_monthly_returns.csv", parse_dates=["Date"])
benchmark = benchmark.rename(columns={"Date": "date", "SPY": "return"})
benchmark = benchmark.set_index("date")["return"]

benchmark_ret = benchmark.reindex(portfolio_returns.index)
benchmark_ret.head()

### 13. Performance Evaluation: Sharpe Ratio & Cumulative Returns

We compute:
- Annualized Sharpe Ratio (computed on **excess returns**)
- Cumulative returns of both the strategy and the SPY benchmark
- A comparison plot

The Sharpe ratio measures **risk-adjusted return**:

\[
Sharpe = \frac{E[R^{excess}]}{\sigma(R^{excess})} \times \sqrt{12}
\]

A higher Sharpe indicates better risk-adjusted performance.

In [ ]:
def sharpe_ratio(r):
    r = pd.Series(r).dropna()
    if r.std() == 0 or len(r) == 0:
        return np.nan
    return (r.mean() / r.std()) * np.sqrt(12)

def annualized_vol(r):
    r = pd.Series(r).dropna()
    if len(r) == 0:
        return np.nan
    return r.std() * np.sqrt(12)

def cagr(r):
    r = pd.Series(r).dropna()
    if len(r) == 0:
        return np.nan
    growth = (1 + r).prod()
    years = len(r) / 12
    if years <= 0:
        return np.nan
    return growth ** (1 / years) - 1

def max_drawdown(cum_curve):
    cum_curve = pd.Series(cum_curve).dropna()
    if len(cum_curve) == 0:
        return np.nan
    running_max = cum_curve.cummax()
    dd = cum_curve / running_max - 1
    return dd.min()

# ---- Portfolio diagnostics ----
holdings_count = (weights_used > 0).sum(axis=1)
turnover = 0.5 * weights_used.diff().abs().sum(axis=1).fillna(0.0)

# Transaction costs are DISABLED by default for this project.
# If you want to enable them later, set cost_bps to a positive value (e.g., 10 or 25).
cost_bps = 0
cost_rate = cost_bps / 10000.0
portfolio_returns_net = portfolio_returns - cost_rate * turnover

# Align RF to portfolio dates for Sharpe and excess-return metrics
rf_series = df_total.set_index("date")["RF"].reindex(portfolio_returns.index)

portfolio_excess = portfolio_returns - rf_series
portfolio_excess_net = portfolio_returns_net - rf_series
benchmark_excess = benchmark_ret - rf_series

# Active return (strategy vs SPY)
active_gross = portfolio_returns - benchmark_ret
active_net = portfolio_returns_net - benchmark_ret

# ---- Equity curves ----
cum_port_gross = (1 + portfolio_returns).cumprod()
cum_port_net = (1 + portfolio_returns_net).cumprod()
cum_bench = (1 + benchmark_ret.fillna(0.0)).cumprod()

# ---- Drawdowns ----
dd_port_gross = cum_port_gross / cum_port_gross.cummax() - 1
dd_port_net = cum_port_net / cum_port_net.cummax() - 1
dd_bench = cum_bench / cum_bench.cummax() - 1

# ---- Summary metrics ----
print("Backtest window:")
print(f"  Start: {portfolio_returns.index.min().date()}")
print(f"  End:   {portfolio_returns.index.max().date()}")
print(f"  Months: {len(portfolio_returns)}")

print("\nHoldings diagnostics:")
print(f"  Avg holdings/month: {holdings_count.mean():.1f}")
print(f"  Min holdings/month: {holdings_count.min()}")
print(f"  Max holdings/month: {holdings_count.max()}")

print("\nTurnover diagnostics:")
print(f"  Avg monthly turnover: {turnover.mean():.3f}")
print(f"  Median monthly turnover: {turnover.median():.3f}")
print(f"  90th pct turnover: {turnover.quantile(0.90):.3f}")

print("\nPerformance (Excess-return metrics for Sharpe):")
print("  Strategy Sharpe (Excess):", sharpe_ratio(portfolio_excess))
print("  Benchmark (SPY) Sharpe (Excess):", sharpe_ratio(benchmark_excess))

print("\nPerformance (Total-return metrics):")
print("  Strategy CAGR:", cagr(portfolio_returns))
print("  SPY CAGR:", cagr(benchmark_ret))

print("  Strategy Vol (ann.):", annualized_vol(portfolio_returns))
print("  SPY Vol (ann.):", annualized_vol(benchmark_ret))

mdd_gross = max_drawdown(cum_port_gross)
mdd_bench = max_drawdown(cum_bench)

print("  Strategy Max Drawdown:", mdd_gross)
print("  SPY Max Drawdown:", mdd_bench)

calmar_gross = cagr(portfolio_returns) / abs(mdd_gross) if pd.notna(mdd_gross) and mdd_gross != 0 else np.nan
calmar_bench = cagr(benchmark_ret) / abs(mdd_bench) if pd.notna(mdd_bench) and mdd_bench != 0 else np.nan

print("  Strategy Calmar:", calmar_gross)
print("  SPY Calmar:", calmar_bench)

print("\nRelative to SPY (Information Ratio on active returns):")
print("  Gross IR:", sharpe_ratio(active_gross))

# ---- Plots (also saved to docs/assets for GitHub Pages) ----
fig = plt.figure(figsize=(12, 5))
plt.plot(cum_port_gross, label="Strategy")
plt.plot(cum_bench, label="SPY")
plt.legend()
plt.title("Cumulative Returns: Strategy vs SPY")
fig.savefig("docs/assets/strategy_cumulative_returns.png", dpi=200, bbox_inches="tight")
plt.show()

fig = plt.figure(figsize=(12, 4))
plt.plot(dd_port_gross, label="Strategy DD")
plt.plot(dd_bench, label="SPY DD")
plt.axhline(0, color="black", linewidth=0.8)
plt.legend()
plt.title("Drawdowns: Strategy vs SPY")
fig.savefig("docs/assets/strategy_drawdowns.png", dpi=200, bbox_inches="tight")
plt.show()

# Rolling 12-month total returns
roll_12m_strategy = (1 + portfolio_returns).rolling(12).apply(np.prod, raw=True) - 1
roll_12m_spy = (1 + benchmark_ret).rolling(12).apply(np.prod, raw=True) - 1

fig = plt.figure(figsize=(12, 4))
plt.plot(roll_12m_strategy, label="Strategy 12M")
plt.plot(roll_12m_spy, label="SPY 12M")
plt.axhline(0, color="black", linewidth=0.8)
plt.legend()
plt.title("Rolling 12-Month Returns")
fig.savefig("docs/assets/strategy_rolling_12m_returns.png", dpi=200, bbox_inches="tight")
plt.show()

### 13A. Prototype Diagnostics (Data Health + Cost Sensitivity + Subperiods)

This section summarizes data coverage, shows baseline performance sensitivity to simple turnover-based transaction cost assumptions, and reports subperiod performance to make regime dependence explicit.

In [ ]:
_realized = df_total.set_index("date")[tickers]
missing_by_ticker = _realized.isna().mean().sort_values(ascending=False)

print("=== DATA HEALTH SUMMARY ===")
print("Tickers in merged dataset:", len(tickers))
print("Months in merged dataset:", df_total["date"].nunique())
print("Overall missing return fraction (mean across tickers):", float(missing_by_ticker.mean()))

print("\nTop 10 tickers by missing return fraction:")
display(missing_by_ticker.head(10).to_frame("missing_frac"))

eligible_universe = _realized.notna().sum(axis=1)
print("\nEligible universe size by month (min/avg/max):",
      int(eligible_universe.min()), float(eligible_universe.mean()), int(eligible_universe.max()))

pred_coverage = pred_df.notna().sum(axis=1)
print("Prediction coverage by month (min/avg/max):",
      int(pred_coverage.min()), float(pred_coverage.mean()), int(pred_coverage.max()))


def _metrics_for_returns(r: pd.Series, rf: pd.Series):
    r = pd.Series(r).dropna()
    if len(r) == 0:
        return {
            "Sharpe (Excess)": np.nan,
            "CAGR": np.nan,
            "Vol (ann.)": np.nan,
            "Max DD": np.nan,
            "Months": 0,
        }

    rf_al = pd.Series(rf).reindex(r.index)
    ex = r - rf_al
    cum = (1 + r).cumprod()

    return {
        "Sharpe (Excess)": sharpe_ratio(ex),
        "CAGR": cagr(r),
        "Vol (ann.)": annualized_vol(r),
        "Max DD": max_drawdown(cum),
        "Months": len(r),
    }


print("\n=== TRANSACTION COST SENSITIVITY (BASELINE) ===")
cost_grid_bps = [0, 10, 25]
rows = []
for bps in cost_grid_bps:
    net = portfolio_returns - (bps / 10000.0) * turnover
    m = _metrics_for_returns(net, rf_series)
    m["cost_bps"] = bps
    rows.append(m)

cost_sens = pd.DataFrame(rows).set_index("cost_bps")
display(cost_sens)


print("\n=== SUBPERIOD PERFORMANCE (GROSS) ===")
subperiods = {
    "2011-2016": ("2011-01-01", "2016-12-31"),
    "2017-2019": ("2017-01-01", "2019-12-31"),
    "2020-2025": ("2020-01-01", str(portfolio_returns.index.max().date())),
}

model_map = {
    "Strategy": portfolio_returns,
    "SPY": benchmark_ret,
}

rows = []
for period, (start, end) in subperiods.items():
    for name, series in model_map.items():
        r = pd.Series(series).loc[start:end]
        rf_al = rf_series.reindex(r.index)
        m = _metrics_for_returns(r, rf_al)
        m["Model"] = name
        m["Period"] = period
        m["Start"] = r.index.min().date() if len(r) else None
        m["End"] = r.index.max().date() if len(r) else None
        rows.append(m)

sub_tbl = pd.DataFrame(rows).set_index(["Period", "Model"])
display(sub_tbl)


## Part III. Two-Model ML Comparison (Linear vs XGBoost)

After establishing the baseline factor strategy, we test **two machine learning models** for cross-sectional top‑50 selection using the same feature set (rolling FF5 betas) and the same walk-forward rules.

- **Model 1 (Linear):** Logistic Regression classifier (linear decision boundary)
- **Model 2 (XGBoost):** Learning-to-rank (non-linear interactions)

**Project narrative:** we tried both ML models and selected the better-performing one based on out-of-sample backtest results (risk-adjusted metrics and robustness checks).

### 13B. Model 2: XGBoost Learning-to-Rank (Top-K Stock Selection)

**Key risks:** overfitting and look-ahead leakage.

**How we avoid look-ahead bias here:**

- Train uses only **past months**.
- Features for month `t` use only information available through **`t-1`**.
- Samples are grouped by month so the model learns **cross-sectional** ranking within each month.

In [ ]:
# XGBoost learning-to-rank (Top-K) — configuration

# XGBRanker uses the scikit-learn API. If scikit-learn is missing, XGBRanker will error.
try:
    import sklearn  # noqa: F401
    _HAS_SKLEARN = True
except Exception:
    _HAS_SKLEARN = False

if not _HAS_XGBOOST:
    _RUN_XGB = False
    print("XGBoost is not installed. To run Model 2, install it with: pip install xgboost==2.0.3")
elif not _HAS_SKLEARN:
    _RUN_XGB = False
    print("scikit-learn is required for Model 2 (XGBRanker). Install it with: pip install scikit-learn==1.4.2")
else:
    _RUN_XGB = True

# Keep this section walk-forward only (no look-ahead)
xgb_k = 50
xgb_n_bins = 5
xgb_min_train_months = 36
xgb_train_window = 60
xgb_retrain_every = 3  # train quarterly to reduce runtime; set to 1 to retrain monthly

xgb_model_params = dict(
    objective="rank:pairwise",
    n_estimators=120,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    verbosity=0,
)

# Placeholders so later cells won't crash if XGBoost isn't available
panel_xgb = None
xgb_feature_cols = None
weights_xgb = None
portfolio_returns_xgb = None
portfolio_returns_xgb_net = None
turnover_xgb = None

In [ ]:
# Build the feature/label panel used by the ML models (XGB ranker + Linear classifier)
# Feature consistency: BOTH models use ONLY the 5 rolling FF5 betas: [MKT, SMB, HML, RMW, CMA]

if _RUN_XGB:
    realized = df_total.set_index("date")[tickers]

    def _stack_to_long(df_wide, value_name):
        s = df_wide.stack(dropna=False).rename(value_name)
        s.index = s.index.set_names(["date", "ticker"])
        return s.reset_index()

    # Betas in long form (betas at date t are estimated using months < t)
    beta_long_parts = []
    for tkr, bdf in beta_dict.items():
        if bdf is None or bdf.empty:
            continue
        tmp = bdf[["MKT", "SMB", "HML", "RMW", "CMA"]].copy()
        tmp["ticker"] = tkr
        tmp["date"] = pd.to_datetime(tmp.index)
        beta_long_parts.append(tmp.reset_index(drop=True))

    beta_long = pd.concat(beta_long_parts, ignore_index=True)

    # Label: excess return in month t (y - RF)
    rf = df_total.set_index("date")["RF"]
    y_excess_wide = realized.sub(rf, axis=0)
    y_excess_long = _stack_to_long(y_excess_wide, "y_excess")

    # Shared panel (one row per date, ticker)
    panel_ml = beta_long.merge(y_excess_long, on=["date", "ticker"], how="left")

    xgb_feature_cols = ["MKT", "SMB", "HML", "RMW", "CMA"]

    panel_ml = panel_ml.replace([np.inf, -np.inf], np.nan)
    panel_ml = panel_ml.dropna(subset=xgb_feature_cols + ["y_excess"]).sort_values(["date", "ticker"]).reset_index(drop=True)

    # Classification label (Linear model): 1 if excess return > 0 else 0
    panel_ml["y_cls"] = (panel_ml["y_excess"] > 0).astype(int)

    # Ranking label (XGBRanker): reserve rel=0 for non-positive excess, bin positives into 1..(B-1)
    panel_xgb = panel_ml.copy()
    B = xgb_n_bins
    is_pos = panel_xgb["y_excess"] > 0
    panel_xgb["rel"] = 0

    pct = panel_xgb.loc[is_pos].groupby("date")["y_excess"].rank(pct=True, method="first")
    panel_xgb.loc[is_pos, "rel"] = (1 + np.floor(pct * (B - 1)).clip(0, B - 2)).astype(int)

    print("panel_ml shape:", panel_ml.shape)
    print("panel_ml months:", panel_ml["date"].nunique())
    print("panel_ml tickers (avg per month):", panel_ml.groupby("date").size().mean())
    panel_ml.head()
else:
    print("Skipping panel build (XGBoost / scikit-learn not available).")

#### 13B.2 Walk-forward training + monthly top-K selection

We train the ranker on a rolling window of past months and then score/rank stocks for the next month.

- Training uses **only months strictly before the test month**.
- Each month is treated as one "query group" for learning-to-rank.
- The output is a dictionary of monthly top-K selections, which we convert into equal weights.

In [ ]:
# Walk-forward Linear classifier (Logistic Regression) using ONLY the same 5 beta features

try:
    from sklearn.linear_model import LogisticRegression
    _HAS_LOGREG = True
except Exception:
    LogisticRegression = None
    _HAS_LOGREG = False

lin_k = 50
lin_min_train_months = 36
lin_train_window = 60
lin_retrain_every = 3

weights_lin = None
portfolio_returns_lin = None

if _RUN_XGB and _HAS_LOGREG and (panel_ml is not None):
    unique_dates = np.array(sorted(panel_ml["date"].unique()))

    lin_selected = {}
    last_model = None
    last_train_end = None

    for i in range(len(unique_dates)):
        dt = unique_dates[i]

        train_end = i  # strictly before dt
        train_start = max(0, train_end - lin_train_window)
        train_dates = unique_dates[train_start:train_end]

        if len(train_dates) < lin_min_train_months:
            continue

        train_df = panel_ml[panel_ml["date"].isin(train_dates)].copy().sort_values(["date", "ticker"])
        test_df = panel_ml[panel_ml["date"] == dt].copy().sort_values(["date", "ticker"])

        if len(test_df) == 0:
            continue

        need_retrain = (
            last_model is None
            or last_train_end is None
            or lin_retrain_every == 1
            or ((train_end - last_train_end) >= lin_retrain_every)
        )

        if need_retrain:
            X_train = train_df[xgb_feature_cols].values
            y_train = train_df["y_cls"].values

            clf = LogisticRegression(
                penalty="l2",
                C=1.0,
                solver="lbfgs",
                max_iter=1000,
            )
            clf.fit(X_train, y_train)
            last_model = clf
            last_train_end = train_end

        X_test = test_df[xgb_feature_cols].values
        p_good = last_model.predict_proba(X_test)[:, 1]  # P(y_excess > 0)

        test_df = test_df.assign(p_good=p_good)
        top = test_df.nlargest(min(lin_k, len(test_df)), "p_good")["ticker"].tolist()
        lin_selected[pd.to_datetime(dt)] = top

    # Convert selections to equal weights
    lin_dates = sorted(lin_selected.keys())
    weights_lin = pd.DataFrame(0.0, index=pd.to_datetime(lin_dates), columns=tickers)

    for dt, names in lin_selected.items():
        if len(names) == 0:
            continue
        weights_lin.loc[dt, names] = 1.0 / len(names)

    # Compute realized portfolio returns (same method as baseline)
    realized_returns_lin = df_total.set_index("date")[tickers]
    common_index = weights_lin.index.intersection(realized_returns_lin.index)

    weights_lin = weights_lin.loc[common_index]
    realized_lin = realized_returns_lin.loc[common_index]

    available = realized_lin.notna()
    w_eff = weights_lin.where(available, 0.0)
    denom = w_eff.sum(axis=1).replace(0.0, np.nan)

    portfolio_returns_lin = (w_eff * realized_lin.fillna(0.0)).sum(axis=1) / denom
    portfolio_returns_lin = portfolio_returns_lin.loc[denom.notna()]

    print("Linear months:", len(portfolio_returns_lin))
    print("Linear start:", portfolio_returns_lin.index.min().date())
    print("Linear end:  ", portfolio_returns_lin.index.max().date())
else:
    print("Skipping linear classifier (requires scikit-learn + panel_ml).")

In [ ]:
# Walk-forward training and portfolio construction

if _RUN_XGB:
    unique_dates = np.array(sorted(panel_xgb["date"].unique()))

    xgb_selected = {}
    last_model = None
    last_train_end = None

    for i in range(len(unique_dates)):
        dt = unique_dates[i]

        train_end = i  # strictly before dt
        train_start = max(0, train_end - xgb_train_window)
        train_dates = unique_dates[train_start:train_end]

        if len(train_dates) < xgb_min_train_months:
            continue

        train_df = panel_xgb[panel_xgb["date"].isin(train_dates)].copy().sort_values(["date", "ticker"])
        test_df = panel_xgb[panel_xgb["date"] == dt].copy().sort_values(["date", "ticker"])

        if len(test_df) == 0:
            continue

        need_retrain = (
            last_model is None
            or last_train_end is None
            or xgb_retrain_every == 1
            or ((train_end - last_train_end) >= xgb_retrain_every)
        )

        if need_retrain:
            grp = train_df.groupby("date").size().values
            X_train = train_df[xgb_feature_cols].values
            y_train = train_df["rel"].values

            ranker = XGBRanker(
                **xgb_model_params,
                eval_metric=f"ndcg@{xgb_k}",
            )

            ranker.fit(X_train, y_train, group=grp)
            last_model = ranker
            last_train_end = train_end

        X_test = test_df[xgb_feature_cols].values
        scores = last_model.predict(X_test)

        test_df = test_df.assign(score=scores)
        top = test_df.nlargest(min(xgb_k, len(test_df)), "score")["ticker"].tolist()
        xgb_selected[pd.to_datetime(dt)] = top

    # Convert selections to equal weights
    xgb_dates = sorted(xgb_selected.keys())
    weights_xgb = pd.DataFrame(0.0, index=pd.to_datetime(xgb_dates), columns=tickers)

    for dt, names in xgb_selected.items():
        if len(names) == 0:
            continue
        weights_xgb.loc[dt, names] = 1.0 / len(names)

    # Compute realized portfolio returns (same method as baseline)
    realized_returns_xgb = df_total.set_index("date")[tickers]
    common_index = weights_xgb.index.intersection(realized_returns_xgb.index)

    weights_xgb = weights_xgb.loc[common_index]
    realized_xgb = realized_returns_xgb.loc[common_index]

    available = realized_xgb.notna()
    w_eff = weights_xgb.where(available, 0.0)
    denom = w_eff.sum(axis=1).replace(0.0, np.nan)

    weights_used_xgb = w_eff.div(denom, axis=0)
    portfolio_returns_xgb = (w_eff * realized_xgb.fillna(0.0)).sum(axis=1) / denom

    active = denom.notna()
    portfolio_returns_xgb = portfolio_returns_xgb.loc[active]
    weights_used_xgb = weights_used_xgb.loc[active]

    turnover_xgb = 0.5 * weights_used_xgb.diff().abs().sum(axis=1).fillna(0.0)
    portfolio_returns_xgb_net = portfolio_returns_xgb - cost_rate * turnover_xgb

    print("XGB months:", len(portfolio_returns_xgb))
    print("XGB start:", portfolio_returns_xgb.index.min().date())
    print("XGB end:  ", portfolio_returns_xgb.index.max().date())
else:
    print("Skipping walk-forward training/backtest (XGBoost / scikit-learn not available).")

#### 13B.3 Model Comparison (Model 1: Linear vs Model 2: XGBoost vs SPY)

For a fair comparison we align all series to the **same XGB backtest window** and compute:
- Sharpe ratio on excess returns
- CAGR on total returns
- A cumulative return plot

In [ ]:
# Comparison report (aligned to XGB window)

if _RUN_XGB and portfolio_returns_xgb is not None and len(portfolio_returns_xgb) > 0:
    aligned_idx = portfolio_returns_xgb.index

    bench_xgb = benchmark.reindex(aligned_idx)
    rf_xgb = df_total.set_index("date")["RF"].reindex(aligned_idx)

    # XGB excess
    xgb_excess = portfolio_returns_xgb - rf_xgb

    # SPY excess
    bench_excess_xgb = bench_xgb - rf_xgb

    print("\n" + "=" * 78)
    print("MODEL COMPARISON (ALIGNED TO XGB WINDOW)")
    print("=" * 78)

    if portfolio_returns_lin is not None and len(portfolio_returns_lin.dropna()) > 0:
        lin_al = portfolio_returns_lin.reindex(aligned_idx)
        lin_excess = lin_al - rf_xgb

        print("\nSharpe (Excess Returns):")
        print("  Linear: ", sharpe_ratio(lin_excess))
        print("  XGB:    ", sharpe_ratio(xgb_excess))
        print("  SPY:    ", sharpe_ratio(bench_excess_xgb))

        print("\nCAGR (Total Returns):")
        print("  Linear: ", cagr(lin_al))
        print("  XGB:    ", cagr(portfolio_returns_xgb))
        print("  SPY:    ", cagr(bench_xgb))

        cum_lin = (1 + lin_al.fillna(0.0)).cumprod()
        cum_xgb = (1 + portfolio_returns_xgb.fillna(0.0)).cumprod()
        cum_spy = (1 + bench_xgb.fillna(0.0)).cumprod()

        fig = plt.figure(figsize=(12, 5))
        plt.plot(cum_lin, label="Linear")
        plt.plot(cum_xgb, label="XGB Ranker")
        plt.plot(cum_spy, label="SPY")
        plt.title("Cumulative Returns: Linear vs XGBoost Ranker vs SPY")
        plt.legend()
        fig.savefig("docs/assets/strategy_cumulative_returns_xgb_compare.png", dpi=200, bbox_inches="tight")
        plt.show()
    else:
        print("\nLinear backtest not available; showing XGB vs SPY only.")

        print("\nSharpe (Excess Returns):")
        print("  XGB: ", sharpe_ratio(xgb_excess))
        print("  SPY: ", sharpe_ratio(bench_excess_xgb))

        print("\nCAGR (Total Returns):")
        print("  XGB: ", cagr(portfolio_returns_xgb))
        print("  SPY: ", cagr(bench_xgb))

        cum_xgb = (1 + portfolio_returns_xgb.fillna(0.0)).cumprod()
        cum_spy = (1 + bench_xgb.fillna(0.0)).cumprod()

        fig = plt.figure(figsize=(12, 5))
        plt.plot(cum_xgb, label="XGB Ranker")
        plt.plot(cum_spy, label="SPY")
        plt.title("Cumulative Returns: XGBoost Ranker vs SPY")
        plt.legend()
        fig.savefig("docs/assets/strategy_cumulative_returns_xgb_compare.png", dpi=200, bbox_inches="tight")
        plt.show()
else:
    print("Skipping comparison (XGBoost backtest not available).")

### 14. Final Comparison: Two ML Models (Linear vs XGBoost) vs SPY

#### Transaction costs (what they were, and why we disabled them)

Earlier, **net returns** were computed by subtracting a simple turnover-based transaction cost:

\[
R^{net}_t = R^{gross}_t - (\text{cost\_rate} \times \text{turnover}_t)
\]

where turnover is computed from changes in portfolio weights.

**For your submission, transaction costs are now set to 0 bps (disabled)** so that all results are purely **gross** and easier to interpret.

(If you ever want to re-enable costs, set `cost_bps` to a positive value in the performance cell.)

#### What this section produces

We compare on a **common aligned window**:
- Summary metrics table
- Calendar-year return table
- Simple comparison plots for:
  - Cumulative returns
  - Drawdowns
  - Rolling 12-month returns

In [ ]:
# All-model comparison tables + simple plots (aligned window)

def _perf_summary_table(model_returns, rf):
    rows = []
    for name, r in model_returns.items():
        r = pd.Series(r).dropna()
        if len(r) == 0:
            continue

        rf_al = rf.reindex(r.index)
        ex = r - rf_al

        cum = (1 + r).cumprod()
        mdd = max_drawdown(cum)
        c = cagr(r)
        vol = annualized_vol(r)
        shrp = sharpe_ratio(ex)
        calmar = c / abs(mdd) if pd.notna(mdd) and mdd != 0 else np.nan

        rows.append(
            {
                "Model": name,
                "Start": r.index.min().date(),
                "End": r.index.max().date(),
                "Months": len(r),
                "Sharpe (Excess)": shrp,
                "CAGR": c,
                "Vol (ann.)": vol,
                "Max DD": mdd,
                "Calmar": calmar,
            }
        )

    out = pd.DataFrame(rows).set_index("Model")
    return out


def _calendar_year_returns(r):
    r = pd.Series(r).dropna()
    if len(r) == 0:
        return pd.Series(dtype=float)
    by_year = (1 + r).groupby(r.index.year).prod() - 1
    return by_year


# Determine common window for comparison (SPY vs Linear vs XGB)
model_series = {
    "SPY": benchmark_ret,
}

if portfolio_returns_lin is not None:
    model_series["Linear"] = portfolio_returns_lin
if portfolio_returns_xgb is not None:
    model_series["XGB"] = portfolio_returns_xgb

common_idx = None
for s in model_series.values():
    idx = pd.Series(s).dropna().index
    common_idx = idx if common_idx is None else common_idx.intersection(idx)

common_idx = common_idx.sort_values()

if len(common_idx) == 0:
    print("No overlapping dates across models; cannot compare.")
else:
    aligned_models = {k: pd.Series(v).reindex(common_idx) for k, v in model_series.items()}
    rf_common = df_total.set_index("date")["RF"].reindex(common_idx)

    # Summary table
    summary = _perf_summary_table(aligned_models, rf_common)
    summary_fmt = summary.copy()

    for col in ["Sharpe (Excess)", "CAGR", "Vol (ann.)", "Max DD", "Calmar"]:
        if col in ["Sharpe (Excess)", "Calmar"]:
            summary_fmt[col] = summary_fmt[col].map(lambda x: f"{x:.3f}" if pd.notna(x) else "")
        else:
            summary_fmt[col] = summary_fmt[col].map(lambda x: f"{x:.2%}" if pd.notna(x) else "")

    print("\n" + "=" * 78)
    print("MODEL SUMMARY (ALIGNED WINDOW)")
    print("=" * 78)
    print(f"Common start: {common_idx.min().date()}  |  Common end: {common_idx.max().date()}  |  Months: {len(common_idx)}")
    display(summary_fmt)

    yr = pd.DataFrame({name: _calendar_year_returns(r) for name, r in aligned_models.items()}).sort_index()
    yr_fmt = yr.applymap(lambda x: f"{x:.2%}" if pd.notna(x) else "")

    print("\nCalendar-year returns (aligned window):")
    display(yr_fmt)

    # ---- Simple comparison plots (3 total) ----
    plot_df = pd.DataFrame(aligned_models).fillna(0.0)

    # Cumulative returns
    cum = (1 + plot_df).cumprod()
    fig = plt.figure(figsize=(12, 5))
    for col in cum.columns:
        plt.plot(cum.index, cum[col], label=col)
    plt.title("Cumulative Returns (Aligned): SPY vs Linear vs XGB")
    plt.legend()
    fig.savefig("docs/assets/compare_cumulative.png", dpi=200, bbox_inches="tight")
    plt.show()

    # Drawdowns
    dd = cum / cum.cummax() - 1
    fig = plt.figure(figsize=(12, 4))
    for col in dd.columns:
        plt.plot(dd.index, dd[col], label=col)
    plt.axhline(0, color="black", linewidth=0.8)
    plt.title("Drawdowns (Aligned): SPY vs Linear vs XGB")
    plt.legend()
    fig.savefig("docs/assets/compare_drawdowns.png", dpi=200, bbox_inches="tight")
    plt.show()

    # Rolling 12-month returns
    roll_12m = (1 + plot_df).rolling(12).apply(np.prod, raw=True) - 1
    fig = plt.figure(figsize=(12, 4))
    for col in roll_12m.columns:
        plt.plot(roll_12m.index, roll_12m[col], label=col)
    plt.axhline(0, color="black", linewidth=0.8)
    plt.title("Rolling 12-Month Returns (Aligned): SPY vs Linear vs XGB")
    plt.legend()
    fig.savefig("docs/assets/compare_rolling_12m.png", dpi=200, bbox_inches="tight")
    plt.show()

In [ ]:
# --- Verification & sanity checks (plain) ---
# Not used to build the strategy; only sanity-check results and guard against look-ahead bias.


def _fmt_pct(x, decimals=2):
    if pd.isna(x):
        return ""
    return f"{x * 100:.{decimals}f}%"


def calendar_year_returns(r):
    r = pd.Series(r).dropna()
    if len(r) == 0:
        return pd.Series(dtype=float)
    return (1 + r).groupby(r.index.year).prod() - 1


def months_per_year(r):
    r = pd.Series(r).dropna()
    if len(r) == 0:
        return pd.Series(dtype=int)
    return r.groupby(r.index.year).size()


print("\n" + "=" * 78)
print("VERIFICATION")
print("=" * 78)

# ---------------------------------------------------------------------
# 1) Calendar-year return verification (Strategy vs SPY)
# ---------------------------------------------------------------------
print("\n" + "-" * 78)
print("1) CALENDAR-YEAR RETURNS (Strategy vs SPY)")
print("-" * 78)

yr_strategy = calendar_year_returns(portfolio_returns)
yr_spy = calendar_year_returns(benchmark_ret)

annual = pd.DataFrame(
    {
        "Months": months_per_year(portfolio_returns),
        "Strategy": yr_strategy,
        "SPY": yr_spy,
    }
).sort_index()

annual["Active"] = annual["Strategy"] - annual["SPY"]

annual_fmt = annual.copy()
for col in ["Strategy", "SPY", "Active"]:
    annual_fmt[col] = annual_fmt[col].map(_fmt_pct)

print(annual_fmt)

full_years = annual[annual["Months"] == 12]

print("\nAverage yearly returns (all years, incl. partial):")
print(f"  Strategy: {annual['Strategy'].mean():.4f}")
print(f"  SPY:      {annual['SPY'].mean():.4f}")

if len(full_years) > 0:
    print("\nAverage yearly returns (FULL years only, 12 months):")
    print(f"  Strategy: {full_years['Strategy'].mean():.4f}")
    print(f"  SPY:      {full_years['SPY'].mean():.4f}")
else:
    print("\nAverage yearly returns (FULL years only): SKIPPED (no complete 12-month years)")

# ---------------------------------------------------------------------
# 2) Look-ahead bias sanity checks
# ---------------------------------------------------------------------
print("\n" + "-" * 78)
print("2) LOOK-AHEAD CHECKS")
print("-" * 78)

# A) Factor forecast lag test
try:
    if "factor_hist" not in globals() or "factor_forecast" not in globals():
        raise RuntimeError("factor_hist / factor_forecast not found")

    nonnull = factor_forecast.dropna()
    if len(nonnull) == 0:
        raise RuntimeError("no non-null forecast months found")

    dt = nonnull.index[0]

    manual_window = factor_hist.loc[:dt].iloc[:-1].tail(36)
    manual_mean = manual_window.mean()
    max_abs_diff = (manual_mean - factor_forecast.loc[dt]).abs().max()

    if max_abs_diff < 1e-12:
        print(f"A) factor_forecast uses shift(1): PASS (checked {dt.date()})")
    else:
        print(f"A) factor_forecast uses shift(1): WARN (max abs diff = {max_abs_diff:.3e})")

except Exception as e:
    print(f"A) factor_forecast lag test: SKIPPED ({e})")

# B) Rolling beta window test (manual recompute for one ticker and one date)
try:
    test_ticker = next((t for t, b in beta_dict.items() if b is not None and not b.empty), None)
    if test_ticker is None:
        raise RuntimeError("no ticker with computed betas found")

    betas = beta_dict[test_ticker]
    test_dt = betas.index[0]

    df_ex_idx = df_excess.set_index("date")

    pos_arr = df_ex_idx.index.get_indexer([pd.Timestamp(test_dt)])
    pos = int(pos_arr[0]) if len(pos_arr) > 0 else -1
    if pos < 0:
        raise RuntimeError(f"could not find beta date {pd.Timestamp(test_dt).date()} in df_excess index")

    window = 36
    min_obs = 24
    if pos < window:
        raise RuntimeError(f"insufficient history at first beta date (pos={pos}, window={window})")

    y_slice = df_ex_idx[test_ticker].astype(float).values[pos - window:pos]
    X_slice = df_ex_idx[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]].astype(float).values[pos - window:pos]
    X_slice = np.column_stack([np.ones(len(X_slice)), X_slice])

    mask = np.isfinite(y_slice) & np.all(np.isfinite(X_slice), axis=1)
    if mask.sum() < min_obs:
        raise RuntimeError(f"not enough finite observations in window (have {mask.sum()}, need {min_obs})")

    beta_manual = ols_numpy(X_slice[mask], y_slice[mask])
    beta_reported = betas.loc[test_dt, ["alpha", "MKT", "SMB", "HML", "RMW", "CMA"]].values

    max_beta_diff = float(np.max(np.abs(beta_manual - beta_reported)))

    if max_beta_diff < 1e-10:
        print(f"B) rolling betas exclude month t in fit: PASS (ticker {test_ticker}, {pd.Timestamp(test_dt).date()})")
    else:
        print(f"B) rolling beta window: WARN (max diff: {max_beta_diff:.3e})")

except Exception as e:
    print(f"B) rolling beta window test: SKIPPED ({e})")

print("C) prediction signal excludes alpha: PASS")
print("\nIf A is PASS and B is PASS (or SKIPPED with a clear reason), you’re consistent with no look-ahead.")

In [ ]:
# --- Simple model summary box (SPY vs Linear vs XGB) ---

# Ensure full column widths are displayed
pd.set_option("display.max_colwidth", None)

def _max_dd_from_returns(r):
    r = pd.Series(r).dropna()
    if len(r) == 0:
        return np.nan
    cum = (1 + r).cumprod()
    return (cum / cum.cummax() - 1).min()


def _summary_row(name, r, rf=None, features="", objective="", selection=""):
    r = pd.Series(r).dropna()
    if len(r) == 0:
        return None

    rf_al = rf.reindex(r.index) if rf is not None else pd.Series(index=r.index, data=0.0)
    ex = r - rf_al

    return {
        "Model": name,
        "Features": features,
        "Objective/Label": objective,
        "Portfolio Rule": selection,
        "Start": r.index.min().date(),
        "End": r.index.max().date(),
        "Months": len(r),
        "Sharpe": sharpe_ratio(ex),
        "CAGR": cagr(r),
        "Max DD": _max_dd_from_returns(r),
    }


rf_series = df_total.set_index("date")["RF"]

rows = []

# SPY benchmark
rows.append(
    _summary_row(
        name="SPY (Benchmark)",
        r=benchmark_ret,
        rf=rf_series,
        features="N/A",
        objective="Passive benchmark (S&P 500 ETF)",
        selection="Buy & Hold",
    )
)

# Linear classifier (Logistic Regression)
if portfolio_returns_lin is not None:
    rows.append(
        _summary_row(
            name="Linear (LogReg)",
            r=portfolio_returns_lin,
            rf=rf_series,
            features="5 rolling FF5 betas: MKT, SMB, HML, RMW, CMA",
            objective="Binary classification: P(y_excess > 0) via Logistic Regression",
            selection="Top-50 by predicted probability, equal weight",
        )
    )

# XGBoost ranker
if portfolio_returns_xgb is not None:
    rows.append(
        _summary_row(
            name="XGBRanker (pairwise)",
            r=portfolio_returns_xgb,
            rf=rf_series,
            features="5 rolling FF5 betas: MKT, SMB, HML, RMW, CMA",
            objective="Pairwise ranking: rel bins from y_excess; bin 0 for y_excess ≤ 0",
            selection="Top-50 by ranking score, equal weight",
        )
    )

summary_box = pd.DataFrame([r for r in rows if r is not None]).set_index("Model")

# Pretty formatting for display
summary_fmt = summary_box.copy()
summary_fmt["Sharpe"] = summary_fmt["Sharpe"].map(lambda x: f"{x:.3f}" if pd.notna(x) else "")
summary_fmt["CAGR"] = summary_fmt["CAGR"].map(lambda x: f"{x:.2%}" if pd.notna(x) else "")
summary_fmt["Max DD"] = summary_fmt["Max DD"].map(lambda x: f"{x:.2%}" if pd.notna(x) else "")

display(summary_fmt)

### 14. Limitations (Important)

- **Survivorship bias**: the universe is based on a current S&P 500 constituents list (not point-in-time membership), which likely biases backtest results upward.
- **Data limitations**: monthly returns reduce noise but also hide intra-month dynamics; delistings and corporate actions may not be fully captured.
- **Transaction costs**: modeled as a simple turnover-based bps cost; real-world costs depend on liquidity, spreads, and market impact.
- **Forecasting**: factor forecasts are a simple rolling mean baseline; more sophisticated forecasts may perform differently.

In [ ]:
# --- Inference (XGB only): recommend tickers for the latest available month ---

infer_k = 50

if not (_RUN_XGB and _HAS_XGBOOST) or panel_xgb is None or xgb_feature_cols is None:
    raise ValueError("XGBoost not enabled or panel_xgb/xgb_feature_cols missing. Run the XGB section first.")

latest_dt = pd.to_datetime(sorted(panel_xgb["date"].unique())[-1])
train_dates = sorted([d for d in panel_xgb["date"].unique() if pd.to_datetime(d) < latest_dt])

print("Inference month:", latest_dt.date())
print("Train months:", len(train_dates), "| Train end:", pd.to_datetime(train_dates[-1]).date())

train_x = panel_xgb[panel_xgb["date"].isin(train_dates)].copy().sort_values(["date", "ticker"])
test_x  = panel_xgb[panel_xgb["date"] == latest_dt].copy().sort_values(["date", "ticker"])

group_train = train_x.groupby("date").size().to_numpy()

xgb_model = XGBRanker(**xgb_model_params)
xgb_model.fit(
    train_x[xgb_feature_cols].values,
    train_x["rel"].values,
    group=group_train,
)

scores = xgb_model.predict(test_x[xgb_feature_cols].values)
xgb_rank = test_x.assign(score=scores).sort_values("score", ascending=False)

xgb_top = xgb_rank.head(min(infer_k, len(xgb_rank)))[["ticker", "score"]].reset_index(drop=True)

print("\nTop tickers to invest in (XGBRanker score):")
display(xgb_top)